In [1]:
!pip install kagglehub
!pip install contractions
!pip install num2words
!pip install inflect
!pip install jieba
!pip install cn2an
!pip install sacrebleu
!pip install safetensors
!pip install datasets

In [2]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import unicodedata
import contractions
import inflect

import num2words
import cn2an
import jieba

import re
from nltk.corpus import stopwords
from num2words import num2words

import pandas as pd

import torch
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from datasets import load_dataset

/opt/anaconda3/envs/telegram-grammar-bot/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from safetensors.torch import load_file

model_name = 'utrobinmv/t5_translate_en_ru_zh_small_1024'
weights_path = '/content/drive/MyDrive/Colab Notebooks/model_weights/model.safetensors'
state_dict = load_file(weights_path)

model = T5ForConditionalGeneration.from_pretrained(model_name,state_dict=state_dict)
tokenizer = T5Tokenizer.from_pretrained(model_name, state_dict=state_dict)

prefix = 'translate to zh: '
src_text = prefix + "1. Яблоко."

# translate Russian to Chinese
input_ids = tokenizer(src_text, return_tensors="pt")

generated_tokens = model.generate(**input_ids)

result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
print(result)
#开发的目的是为用户提供个人同步翻译。

In [ ]:
import re
import jieba

class CleanChineseText:
    def __init__(self, zh_text):
        self.zh_text = zh_text
    def escape_en_char(self):
        en_pattern = "[a-zA-Z]+"
        self.zh_text = re.sub(en_pattern, ' ', self.zh_text)
        return self
    def remove_enter_and_symbols(self):
        self.zh_text = re.sub(r'^\d+\.', '', self.zh_text).strip()
        return self
    def remove_gaps(self):
        self.zh_text = self.zh_text.replace(' ', '')
        return self
    def remove_punct(self):
        self.zh_text = self.zh_text.strip('，')
        return self

    def clean_all(self):
        self.remove_punct()
        self.remove_enter_and_symbols()
        self.escape_en_char()
        self.remove_gaps()
        return self.zh_text

In [ ]:

class CleanRussianText:
    def __init__(self, ru_text):
        self.ru_text = ru_text
    # def to_one_style(self):
    #     # Lower case
    #     self.ru_text = self.ru_text.lower()
    #     return self
    # def remove_numbers_enters_symbols(self):
    #     # remove all symbols, numbers and /n in the end of the strings
    #     self.ru_text = re.sub(r'[^\w\s]', '', self.ru_text).strip()
    #     # self.ru_text = re.sub(r'^\d+\.', '', self.ru_text).strip()
    #     # self.ru_text = re.sub('[^а-яА-Я0-9\s]', ' ', self.ru_text)
    #     return self

    # def combine_numbers(self):
    #     # combine numbers with a space, such as '40 000' to '40000'
    #     self.ru_text = re.sub(r'(\d+)\s*(\d+)', r'\1\2', self.ru_text)
    #     return self
    # def separate_numbers_and_words(self):
    #     # separate words like '11летнему' to '11 летнему'
    #     self.ru_text = re.sub(r'(\d+)([а-яА-Я]+)', r'\1 \2', self.ru_text)
    #     return self

    # def numbers_to_words(self):
    #     # convert numbers to words '40000' to 'сорок тысяч'
    #     def convert_numbers(match):
    #       return num2words(int(match.group()), lang='ru', to='ordinal')
    #     self.ru_text = re.sub(r'\d+', convert_numbers, self.ru_text)
    #     return self

    def clean_all(self):
      self.ru_text = self.ru_text.lower().strip()
      self.ru_text = re.sub(r'\s+', ' ', self.ru_text)
      self.ru_text = re.sub(r'[^\w\s]', '', self.ru_text)
      self.ru_text = re.sub(r'(\d+)\s*(\d+)', r'\1\2', self.ru_text)
      # self.combine_numbers()
      # self.separate_numbers_and_words()
      # self.numbers_to_words()
      return self.ru_text

In [ ]:
text = ' ♫ как лёд сходит и 3 000 лет идет к 11летнему Тиму ♫'
cleaner = CleanRussianText(text)
result = cleaner.clean_all()
print(result)

In [ ]:
text = '，我是1个anzh很好好的15岁的孩子，'
cleaner = CleanChineseText()
result = cleaner.clean_all()
print(result)

In [ ]:
def tokenize_dataset(source_ru, target_zh, tokenizer=tokenizer):
    prefix = 'translate to zh'
    source_ru_texts = [f'{prefix}: {text}' for text in source_ru]
    tokenized_ru_texts = [tokenizer(text, truncation=True, padding=True, return_tensors='pt') for text in source_ru_texts]

    target_zh_texts = [tokenizer(zh_text, truncation=True, padding=True, return_tensors='pt') for zh_text in target_zh]
    return tokenized_ru_texts, target_zh_texts

class TokenizedT5CustomDataset(Dataset):
    def __init__(self, tokenized_ru_texts_ids, target_zh_texts_ids):
        self.source_ru = tokenized_ru_texts_ids
        self.target_zh = target_zh_texts_ids

    def __getitem__(self, idx):
        return {
            'input_ids': self.source_ru[idx]['input_ids'].squeeze(0),
            'attention_mask': self.source_ru[idx]['attention_mask'].squeeze(0),
            'labels': self.target_zh[idx]['input_ids'].squeeze(0)
        }
    def __len__(self):
        return len(self.source_ru)

In [ ]:
subtitle_dataset = load_dataset("open_subtitles", lang1="ru", lang2="zh_cn", trust_remote_code=True)

In [ ]:
df = pd.DataFrame(subtitle_dataset['train'])

In [ ]:
df['cleaned_ru'] = df['translation'].apply(lambda x: CleanRussianText(x['ru']).clean_all())

In [ ]:
df['cleaned_zh'] = df['translation'].apply(lambda x: CleanChineseText(x['zh_cn']).clean_all())

In [ ]:
df['translation'][934]

In [ ]:
from sklearn.model_selection import train_test_split
source_ru_train_subt, source_ru_val_subt, target_zh_train_subt, target_zh_val_subt = train_test_split(
    df['cleaned_ru'], df['cleaned_zh'], test_size=0.2, random_state=42)

In [ ]:
tokenized_ru_texts_train, target_zh_texts_train = tokenize_dataset(source_ru_train_subt, target_zh_train_subt)
tokenized_ru_texts_val, target_zh_texts_val = tokenize_dataset(source_ru_val_subt, target_zh_val_subt)

In [ ]:
print(tokenized_ru_texts_train[:10])

In [ ]:
print(target_zh_texts_train[:10])

In [ ]:
print(tokenized_ru_texts_val[:10])

In [ ]:
print(target_zh_texts_val[:10])

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

train_dataset = TokenizedTedCustomDataset(tokenized_ru_texts_ids=tokenized_ru_texts_train, target_zh_texts_ids=target_zh_texts_train)
test_dataset = TokenizedTedCustomDataset(tokenized_ru_texts_ids=tokenized_ru_texts_val, target_zh_texts_ids=target_zh_texts_val)

train_loader = DataLoader(dataset=train_dataset, batch_size=256, shuffle=False, collate_fn=data_collator)
test_loader = DataLoader(dataset=test_dataset, batch_size=256, shuffle=False, collate_fn=data_collator)

In [ ]:
test_dataset[0]

In [ ]:
from nltk.translate.bleu_score import corpus_bleu

def bleu_score(model, tokenizer, eval_pred_loader):
  actual, predicted = [], []
  model.to('cuda')
  model.eval()
  with torch.no_grad():

    for batch in eval_pred_loader:
      input_ids = batch['input_ids'].to('cuda')
      attention_mask = batch['attention_mask'].to('cuda')
      labels = batch['labels'].to('cuda')

      outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask)

      decoded_labels = tokenizer.batch_decode(
          [[token if token != -100 else tokenizer.pad_token_id for token in label] for label in labels],
          skip_special_tokens=True,
      )

      predictions = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

      actual.append([decoded_labels[0]])
      predicted.append(predictions[0])

  bleu_dic = {}
  bleu_dic['bleu'] = corpus_bleu(actual, predicted)
  # bleu_dic['1-2-grams'] = corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing)
  # bleu_dic['1-3-grams'] = corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0), smoothing_function=smoothing)
  # bleu_dic['1-4-grams'] = corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smoothing)

  return bleu_dic

In [ ]:
bleu_test_subt = bleu_score(model, tokenizer, test_loader)
print(bleu_test_subt)

In [ ]:
from transformers import Trainer
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir='/result',
    evaluation_strategy='steps',
    per_device_train_batch_size=512,
    per_device_eval_batch_size=512,
    learning_rate=5e-05,
    num_train_epochs=10,
    logging_steps=1000,
    save_steps=1000,
    save_total_limit=2,
    seed=42,
    logging_dir="./logs",
    fp16=True,
    report_to="none"

)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

In [ ]:
bleu_test_subt_after_test = bleu_score(model, tokenizer, test_loader)
print(bleu_test_subt_after_test)